In [1]:
from photerr import LsstErrorModel
import pandas as pd
import numpy as np

In [2]:
noiseless_photometry = np.load("/Users/fpetri/repos/LBGforecast/data/data/training_data.npy")[:1000000, :6]
sps_params = np.load("/Users/fpetri/repos/LBGforecast/data/data/training_params.npy")[:1000000, :]
df = pd.DataFrame(noiseless_photometry, columns=['u', 'g', 'r', 'i', 'z', 'y'])
df

,u,g,r,i,z,y
0,35.979999,34.826707,33.827043,30.591961,29.763766,24.299834
1,20.662052,20.634097,20.163877,20.307578,20.376406,20.447159
2,27.477763,26.960422,26.893122,26.893755,26.835643,26.860594
3,36.451890,35.415322,34.327853,32.541182,31.546398,30.657177
4,18.311344,18.285075,18.000323,18.051494,18.085408,18.033818
...,...,...,...,...,...,...
999995,40.123789,38.240046,27.177094,24.436528,23.433175,22.784754
999996,18.016546,17.793790,17.892474,17.391550,17.905033,18.120327
999997,28.242125,23.668817,22.560040,21.593582,20.983335,20.537410
999998,43.036269,37.746546,29.504046,27.274358,26.449181,26.099206


In [3]:
import lbg_forecast.noise as no

u, g, r = no.get_noisy_magnitudes(sps_params, noiseless_photometry, brightness_cut=18,)

r[1].shape

(54047, 6)

In [4]:
errModel5sig = LsstErrorModel(sigLim=5)
errModel5sig.getLimitingMags()

{'u': 26.0374014480827,
 'g': 27.29469867039545,
 'r': 27.31270169467983,
 'i': 26.868862973183287,
 'z': 26.22934045752097,
 'y': 25.299990902598516}

In [5]:
catalog_with_errors5sig = errModel5sig(df, random_state=42).filter(['u', 'g', 'r', 'i', 'z', 'y'])
catalog_with_errors5sig

,u,g,r,i,z,y
0,inf,inf,inf,inf,inf,24.419026
1,20.661355,20.635699,20.163962,20.311902,20.371887,20.442743
2,inf,26.793837,26.826197,inf,inf,inf
3,inf,inf,inf,inf,inf,inf
4,18.313505,18.286843,17.997658,18.049663,18.083339,18.031653
...,...,...,...,...,...,...
999995,inf,inf,inf,24.460094,23.437816,22.793269
999996,18.015375,17.789425,17.888425,17.395390,17.902992,18.127446
999997,inf,23.677204,22.562164,21.599041,20.988389,20.538090
999998,inf,inf,inf,inf,inf,inf


In [6]:
errModel2sig = LsstErrorModel(sigLim=2)
errModel2sig.getLimitingMags(nSigma=2)

{'u': 27.034728204977878,
 'g': 28.29071405333543,
 'r': 28.308402890903814,
 'i': 27.86456416940727,
 'z': 27.225085676404127,
 'y': 26.295736121481674}

In [7]:
catalog_with_errors2sig = errModel2sig(df, random_state=42).filter(['u', 'g', 'r', 'i', 'z', 'y'])
catalog_with_errors2sig.rename(columns={"u": "u2", "g": "g2", "r": "r2", "i": "i2", "z": "z2", "y": "y2" }, inplace=True)
catalog_with_errors2sig

,u2,g2,r2,i2,z2,y2
0,inf,inf,inf,inf,inf,24.419026
1,20.661355,20.635699,20.163962,20.311902,20.371887,20.442743
2,inf,26.793837,26.826197,27.103724,26.704071,inf
3,inf,inf,inf,inf,inf,inf
4,18.313505,18.286843,17.997658,18.049663,18.083339,18.031653
...,...,...,...,...,...,...
999995,inf,inf,27.325318,24.460094,23.437816,22.793269
999996,18.015375,17.789425,17.888425,17.395390,17.902992,18.127446
999997,inf,23.677204,22.562164,21.599041,20.988389,20.538090
999998,inf,inf,inf,inf,26.507004,25.681328


In [8]:
brightness_cut = 18
siglim = errModel5sig.getLimitingMags()
catalog = catalog_with_errors5sig.join(catalog_with_errors2sig)

for column in ['u','g','r','i','z','y']:
    catalog = catalog.drop(catalog[catalog[column] < brightness_cut].index)

In [9]:
udrop = catalog.replace([np.inf, -np.inf], np.nan, inplace=False).dropna(axis=0, subset=['g', 'r']).filter(['u','g','r','i','z','y'])
udrop = udrop.replace([np.nan], siglim['u'], inplace=False)
udrop

,u,g,r,i,z,y
1,20.661355,20.635699,20.163962,20.311902,20.371887,20.442743
2,26.037401,26.793837,26.826197,26.037401,26.037401,26.037401
11,22.876032,22.003866,20.548669,21.238673,20.012281,21.216944
15,26.037401,27.140973,25.679089,24.503926,23.591683,25.233829
16,26.006904,24.272420,23.578712,23.257801,23.050582,22.889766
...,...,...,...,...,...,...
999984,26.037401,26.243390,25.657850,25.153262,25.333205,23.024671
999993,26.037401,27.287871,26.684982,25.835360,24.417397,24.026091
999994,22.472164,21.962778,21.511762,20.850805,20.969838,20.648415
999997,26.037401,23.677204,22.562164,21.599041,20.988389,20.538090


In [10]:
gdrop = catalog.replace([np.inf, -np.inf], np.nan, inplace=False).dropna(axis=0, subset=['r', 'i']).filter(['u2','g','r','i','z','y'])
gdrop = gdrop.drop(gdrop[np.isnan(gdrop.u2) == False].index)
gdrop = gdrop.replace([np.nan], siglim['g'], inplace=False)
gdrop

,u2,g,r,i,z,y
9,27.294699,27.294699,26.678613,25.269228,24.819414,24.586305
15,27.294699,27.140973,25.679089,24.503926,23.591683,25.233829
22,27.294699,27.294699,25.829620,22.948710,22.547465,22.218772
24,27.294699,27.294699,26.902024,24.374690,24.538475,24.145958
32,27.294699,27.024264,25.528666,24.899056,24.412637,24.276253
...,...,...,...,...,...,...
999972,27.294699,24.993288,22.928846,22.599983,22.469577,22.408749
999977,27.294699,22.905594,21.386892,21.327301,21.338190,21.324447
999991,27.294699,27.294699,27.151838,26.205105,25.286644,24.689674
999993,27.294699,27.287871,26.684982,25.835360,24.417397,24.026091


In [11]:
rdrop = catalog.replace([np.inf, -np.inf], np.nan, inplace=False).dropna(axis=0, subset=['r', 'i', 'z']).filter(['u','g2','r','i','z','y'])
rdrop = rdrop.drop(rdrop[np.isnan(rdrop.g2) == False].index)
rdrop = rdrop.replace([np.nan], siglim['r'], inplace=False)
rdrop

,u,g2,r,i,z,y
9,27.312702,27.312702,26.678613,25.269228,24.819414,24.586305
22,27.312702,27.312702,25.829620,22.948710,22.547465,22.218772
24,27.312702,27.312702,26.902024,24.374690,24.538475,24.145958
39,27.312702,27.312702,27.291335,25.421941,25.213747,24.992791
49,27.312702,27.312702,26.402096,25.927107,25.615316,25.050883
...,...,...,...,...,...,...
999907,27.312702,27.312702,27.083355,23.760906,22.883012,22.086750
999924,27.312702,27.312702,26.282670,24.577204,24.001935,23.760637
999954,27.312702,27.312702,21.704154,18.264354,18.152294,18.149710
999956,27.312702,27.312702,27.303187,26.239816,25.678586,27.312702


In [12]:
u_dropouts = udrop.to_numpy()
g_dropouts = gdrop.to_numpy()
r_dropouts = rdrop.to_numpy()

In [13]:
u_index = udrop.index.to_numpy()
g_index = gdrop.index.to_numpy()
r_index = rdrop.index.to_numpy()